In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
import movie_stars
import warnings
warnings.filterwarnings("ignore")
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')
df_sos = pd.read_csv('/Users/cam/Downloads/Tsantaki_SoS_291021_CDS.csv')


In [2]:
df_open_clusters = pd.read_csv('/Users/cam/Downloads/open_clusters_3d.csv')
df_oc_full = pd.read_csv('/Users/cam/Downloads/open_clusters_full.csv')


df_oc_full = pd.merge(left = df_oc_full, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id')
df_oc_full = pd.merge(left = df_oc_full, right = df_sos, how = 'left', left_on = 'source_id', right_on = 'gaiaSourceId')



In [3]:
df_oc_apogee = df_oc_full.loc[df_oc_full['vhelio_avg'].notnull()]
df_oc_apogee['rv_chosen'] = df_oc_apogee['vhelio_avg']

df_oc_sos = df_oc_full.loc[(df_oc_full['RVcor_merged'].notnull()) & (df_oc_full['vhelio_avg'].isnull())]
df_oc_sos['rv_chosen'] = df_oc_sos['RVcor_merged']

df_oc_gaia = df_oc_full.loc[(df_oc_full['rv'].notnull()) & (df_oc_full['vhelio_avg'].isnull()) & (df_oc_full['RVcor_merged'].isnull())]
df_oc_gaia['rv_chosen'] = df_oc_gaia['rv']

df_oc_no_rv = df_oc_full.loc[(df_oc_full['rv'].isnull()) & (df_oc_full['vhelio_avg'].isnull()) & (df_oc_full['RVcor_merged'].isnull())]

df_oc_full = pd.concat([df_oc_apogee, df_oc_sos, df_oc_gaia, df_oc_no_rv])


In [4]:
oc_grouped_df_list = []
oc_groups = df_oc_full.groupby('cluster')
for cluster, df_oc_grouped in oc_groups:
    df_oc_grouped = df_oc_grouped.rename(columns = {'ra_x' : 'ra_med', 
                                                    'dec' : 'dec_med', 
                                                    'parallax' : 'parallax_med', 
                                                    'pmra' : 'pmra_med',
                                                    'pmdec' : 'pmdec_med',
                                                    'rv_chosen' : 'rv_chosen_med'
                                                    })
    df_oc_grouped = df_oc_grouped.loc[df_oc_grouped['rv_chosen_med'].notnull()]
    #if len(df_oc_grouped) > 5:
        #df_oc_grouped = df_oc_grouped.median()
    df_oc_grouped['cluster'] = cluster
    df_oc_grouped['n'] = len(df_oc_grouped)
    df_oc_grouped['n_rv'] = len(df_oc_grouped.loc[df_oc_grouped.rv_chosen_med.notnull()])
    oc_grouped_df_list.append(df_oc_grouped)

df_oc_grouped_full = pd.concat(oc_grouped_df_list)
df_oc_grouped_full = df_oc_grouped_full.groupby('cluster').median().reset_index()

df_open_clusters = pd.merge(left = df_open_clusters, right = df_oc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
df_open_clusters.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_open_clusters.to_csv('/Users/cam/Downloads/cg_ocs_full.csv', index = False)

In [5]:
# crossmatch Sebastian catalog
df_sc = pd.read_csv('/Users/cam/Downloads/sco_cen_full.csv')
filter_col = [col for col in df_sc if col.startswith('cluster')]
sc_clusters_list = []
for c in filter_col:
    df_in_cluster = df_sc.loc[df_sc[c] == True]
    df_in_cluster['label'] = c
    sc_clusters_list.append(df_in_cluster)
df_sc = pd.concat(sc_clusters_list)

df_sc_apogee_xmatch = pd.merge(left = df_sc, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id', suffixes = ('', '_y'))
df_sc_full = pd.merge(left = df_sc_apogee_xmatch, right = df_sos, how = 'left', left_on = 'source_id', right_on = 'gaiaSourceId', suffixes = ('', '_y'))

In [6]:
df_sc_apogee = df_sc_full.loc[df_sc_full['vhelio_avg'].notnull()]
df_sc_apogee['rv_chosen'] = df_sc_apogee['vhelio_avg']

df_sc_sos = df_sc_full.loc[(df_sc_full['RVcor_merged'].notnull()) & (df_sc_full['vhelio_avg'].isnull())]
df_sc_sos['rv_chosen'] = df_sc_sos['RVcor_merged']

df_sc_gaia = df_sc_full.loc[(df_sc_full['dr2_radial_velocity'].notnull()) & (df_sc_full['vhelio_avg'].isnull()) & (df_sc_full['RVcor_merged'].isnull())]
df_sc_gaia['rv_chosen'] = df_sc_gaia['dr2_radial_velocity']

df_sc_no_rv = df_sc_full.loc[(df_sc_full['dr2_radial_velocity'].isnull()) & (df_sc_full['vhelio_avg'].isnull()) & (df_sc_full['RVcor_merged'].isnull())]

df_sc_full = pd.concat([df_sc_apogee, df_sc_sos, df_sc_gaia, df_sc_no_rv])

In [7]:
sc_grouped_df_list = []
sc_groups = df_sc_full.groupby('label')
for cluster, df_sc_grouped in sc_groups:
    df_sc_grouped = df_sc_grouped.rename(columns = {'ra_x' : 'ra'})
    df_sc_grouped = df_sc_grouped.loc[df_sc_grouped['rv_chosen'].notnull()]
    if len(df_sc_grouped) > 20:
        #df_oc_grouped = df_oc_grouped.median()
        df_sc_grouped['label'] = cluster
        df_sc_grouped['n'] = len(df_sc_grouped)
        sc_grouped_df_list.append(df_sc_grouped)

df_sc_grouped_full = pd.concat(sc_grouped_df_list)
df_sc_grouped_full = df_sc_grouped_full.groupby('label').median().reset_index()

#df_sc_out = pd.merge(left = df_sc_full, right = df_sc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
#df_sc_out.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_sc_grouped_full.to_csv('/Users/cam/Downloads/sc_sebastian_full.csv', index = False)

In [8]:
hdu_hao_full = fits.open('/Users/cam/Downloads/asu-34.fit')
d = hdu_hao_full[1].data
df_hao_full = pd.DataFrame({'ra' : d['RA_ICRS'].byteswap().newbyteorder(),
                            'dec' : d['DE_ICRS'].byteswap().newbyteorder(), 
                            'parallax' : d['plx'].byteswap().newbyteorder(), 
                            'pmra' : d['pmRA'].byteswap().newbyteorder(), 
                            'pmdec' : d['pmDE'].byteswap().newbyteorder(), 
                            'radial_velocity' : d['RV'].byteswap().newbyteorder(), 
                            'Cluster' : d['Cluster'].byteswap().newbyteorder(),
                            'source_id' : d['GaiaEDR3'].byteswap().newbyteorder(),
                            'Gmag' : d['Gmag'].byteswap().newbyteorder(),
                            'bp_rp' : d['BP-RP'].byteswap().newbyteorder()
})

df_hao_apogee = pd.merge(left = df_hao_full, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id', suffixes=('', '_y'))
df_hao_full = pd.merge(left = df_hao_apogee, right = df_sos, how = 'left', left_on = 'source_id', right_on = 'gaiaSourceId', suffixes=('', '_y'))



In [9]:
df_hao_apogee = df_hao_full.loc[df_hao_full['vhelio_avg'].notnull()]
df_hao_apogee['rv_chosen'] = df_hao_apogee['vhelio_avg']

df_hao_sos = df_hao_full.loc[(df_hao_full['RVcor_merged'].notnull()) & (df_hao_full['vhelio_avg'].isnull())]
df_hao_sos['rv_chosen'] = df_hao_sos['RVcor_merged']

df_hao_gaia = df_hao_full.loc[(df_hao_full['radial_velocity'].notnull()) & (df_hao_full['vhelio_avg'].isnull()) & (df_hao_full['RVcor_merged'].isnull())]
df_hao_gaia['rv_chosen'] = df_hao_gaia['radial_velocity']

df_hao_no_rv = df_hao_full.loc[(df_hao_full['radial_velocity'].isnull()) & (df_hao_full['vhelio_avg'].isnull()) & (df_hao_full['RVcor_merged'].isnull())]
df_hao_no_rv['rv_chosen'] = [np.nan]*len(df_hao_no_rv)

df_hao_full = pd.concat([df_hao_apogee, df_hao_sos, df_hao_gaia, df_hao_no_rv])
df_hao_full
df_hao_grouped = pd.read_csv('/Users/cam/Downloads/hao_grouped.csv')
clusters = df_hao_full.groupby(['Cluster'])

clusters_list = []
for c, df_c in clusters:
    df_c['age'] = df_hao_grouped.loc[df_hao_grouped.Cluster == c].age.values*np.ones(len(df_c))
    clusters_list.append(df_c)
df_hao_full = pd.concat(clusters_list)


df_hao_full.to_csv('/Users/cam/Downloads/hao_oc_rv_full.csv', index=False)

In [19]:
hdu_he_full = fits.open('/Users/cam/Downloads/He_OCs_full.fit')
d = hdu_he_full[1].data
df_he_full = pd.DataFrame(d)


df_he_apogee = pd.merge(left = df_he_full, right = df_apogee, how = 'left', left_on = 'GaiaEDR3', right_on = 'gaiaedr3_source_id', suffixes=('', '_y'))
df_he_full = pd.merge(left = df_he_apogee, right = df_sos, how = 'left', left_on = 'GaiaEDR3', right_on = 'gaiaSourceId', suffixes=('', '_y'))

In [36]:

df_he_apogee = df_he_full.loc[df_he_full['vhelio_avg'].notnull()]
df_he_apogee['rv_chosen'] = df_he_apogee['vhelio_avg']

df_he_sos = df_he_full.loc[(df_he_full['RVcor_merged'].notnull()) & (df_he_full['vhelio_avg'].isnull())]
df_he_sos['rv_chosen'] = df_he_sos['RVcor_merged']

df_he_gaia = df_he_full.loc[(df_he_full['RV'] != 0.) & (df_he_full['vhelio_avg'].isnull()) & (df_he_full['RVcor_merged'].isnull())]
df_he_gaia['rv_chosen'] = df_he_gaia['RV']

df_he_no_rv = df_he_full.loc[(df_he_full['RV'] == 0.) & (df_he_full['vhelio_avg'].isnull()) & (df_he_full['RVcor_merged'].isnull())]
df_he_no_rv['rv_chosen'] = [np.nan]*len(df_he_no_rv)

df_he_full = pd.concat([df_he_apogee, df_he_sos, df_he_gaia, df_he_no_rv])

hdu_he_grouped = fits.open('/Users/cam/Downloads/He_OCs.fit')
df_he_grouped = pd.DataFrame(hdu_he_grouped[1].data)
clusters = df_he_full.groupby(['Cluster'])

clusters_list = []
for c, df_c in clusters:
    df_c['n'] = np.ones(len(df_c))*len(df_c)
    df_c['n_rv'] = len(df_c.loc[df_c['rv_chosen'].notnull()])*np.ones(len(df_c))
    df_c['age'] = (10**(df_he_grouped.loc[df_he_grouped.Cluster == c]['logAge'].values)/1e6)*np.ones(len(df_c))
    clusters_list.append(df_c)
df_he_full = pd.concat(clusters_list)


df_he_full.to_csv('/Users/cam/Downloads/he_oc_rv_full.csv', index=False)

In [38]:
df_he_full.Cluster

143        1
0          1
1          1
2          1
3          1
        ... 
66463    541
66464    541
66465    541
66466    541
66467    541
Name: Cluster, Length: 66468, dtype: int32